In [1]:
import os, sys
import django
import pytz
from dateutil import parser
from django.utils.timezone import make_aware

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "tshilo_dikotla.settings"

# Connect to Django ORM
django.setup()
import uuid
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

try:
    td = Site.objects.get(id=40)
except Site.DoesNotExist:
    td = Site.objects.create(id=40, domain='td-live.bhp.org.bw', name='td-live.bhp.org.bw')
path = '/Users/coulsonkgathi/source/td_data_files/20190707/'

In [2]:
# Subject Screening
from td_maternal.models import SubjectScreening
from td_maternal.identifiers import ScreeningIdentifier
from django.contrib.sites.models import Site

fname = path + 'maternal_eligibility.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sc_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        raise ValidationError('Line is not the right size')
    sc_data.append(data)

site = Site.objects.get(id=40)
count = 0
already_created = 0
for sc in sc_data:
    del sc['_state']
    del sc['registered_subject_id']
    sc.update(
        report_datetime=make_aware(parser.parse(sc.get('report_datetime'))),
        created=make_aware(parser.parse(sc.get('created'))),
        site=site,
        id=uuid.UUID(sc.get('id'))
    )
    del sc['eligibility_id']
    del sc['export_change_type']
    del sc['export_uuid']
    del sc['exported']
    del sc['exported_datetime']
    if sc['modified']:
        sc.update(modified=make_aware(parser.parse(sc.get('modified'))))
    else:
        sc.update(modified=None)
    try:
        SubjectScreening.objects.get(id=sc.get('id'))
    except SubjectScreening.DoesNotExist:
        sc.update(screening_identifier=ScreeningIdentifier().identifier)
        subject_Screening = SubjectScreening(**sc)
        subject_Screening.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created subject screening {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sc_data))
print("Existing: ", SubjectScreening.objects.all().count())

Total created subject screening 555
Already created 0
Expected:  555
Existing:  555


In [3]:
# TD Consent Version
import uuid
from td_maternal.models import TdConsentVersion
from django.core.exceptions import ValidationError

fname = path + 'td_consent_version.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

td_consent_version_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    td_consent_version_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in td_consent_version_data:
    del dd['_state']
    created = parser.parse(dd.get('created'))
    created = make_aware(created)
    dd.update(
        created=created,
        report_datetime=make_aware(parser.parse(dd.get('report_datetime'))),
        id=uuid.UUID(dd.get('id'))
    )
    maternal_eligibility_id = uuid.UUID(dd.get('maternal_eligibility_id'))
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = make_aware(modified)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    try:
        subject_screening = SubjectScreening.objects.get(id=maternal_eligibility_id)
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Subject Screening can not be missing ', maternal_eligibility_id)
    else:
        del dd['maternal_eligibility_id']
        screening_identifier = subject_screening.screening_identifier
        try:
            TdConsentVersion.objects.get(screening_identifier=screening_identifier)
        except TdConsentVersion.DoesNotExist:
            dd.update(screening_identifier=screening_identifier)
            td_consent_version = TdConsentVersion(**dd)
            td_consent_version.save_base(raw=True)
            count += 1
        else:
            already_created += 1
print(f'Total created td_consent_version {count}')
print(f'Already created td_consent_version {already_created}')
print("Expected: ", len(td_consent_version_data))
print("Existing: ", TdConsentVersion.objects.all().count())

Total created td_consent_version 458
Already created td_consent_version 0
Expected:  458
Existing:  458


In [4]:
# Subject Consent
import uuid
from td_maternal.models import SubjectConsent
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

fname = path + 'maternal_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

s_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    s_consent_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for dd in s_consent_data:
    del dd['_state']
    del dd['export_change_type']
    del dd['export_uuid']
    del dd['exported']
    del dd['exported_datetime']
    del dd['study_site']
    dd.update(site=td)
    created = parser.parse(dd.get('created'))
    created = make_aware(created)
    consent_datetime = parser.parse(dd.get('consent_datetime'))
    consent_datetime = make_aware(consent_datetime)
    dob = make_aware(parser.parse(dd.get('dob')))
    dob = dob.date()
    dd.update(
        created=created,
        consent_datetime=consent_datetime,
        report_datetime=make_aware(parser.parse(dd.get('consent_datetime'))),
        dob=dob,
        id=uuid.UUID(dd.get('id'))
    )
    if dd['subject_identifier_as_pk']:
        dd.update(
            subject_identifier_as_pk=uuid.UUID(dd.get('subject_identifier_as_pk')))
    else:
        dd.update(
            subject_identifier_as_pk=dd.get('id'))
    if dd['is_verified_datetime']:
        is_verified_datetime = parser.parse(dd.get('is_verified_datetime'))
        is_verified_datetime = make_aware(is_verified_datetime)
        dd.update(
        is_verified_datetime=is_verified_datetime)
    else:
        dd.update(is_verified_datetime=None)
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        modified = make_aware(modified)
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    if dd['is_verified'] == 'TRUE':
        dd.update(is_verified=True)
    elif dd['is_verified'] == 'FALSE':
        dd.update(is_verified=False)
    if dd['guardian_name']:
        guardian_name = dd['guardian_name']
        name1, name2 = guardian_name.split(',')
        guardian_name = name1 + ', ' + name2
        dd.update(guardian_name=guardian_name)
    try:
        subject_screening = SubjectScreening.objects.get(id=dd['maternal_eligibility_id'])
    except SubjectScreening.DoesNotExist:
        raise ValidationError('Eligibility can not be missing for this subject', dd['subject_identifier'])
    else:
        dd.update(screening_identifier=subject_screening.screening_identifier)
        del dd['maternal_eligibility_id']
    try:
        SubjectConsent.objects.get(id=dd.get('id'))
    except SubjectConsent.DoesNotExist:
        subject_consent = SubjectConsent(**dd)
        subject_consent.save_base(raw=True)
        count += 1
        subject_screening.subject_identifier = dd['subject_identifier']
        subject_screening.save_base(raw=True)
    else:
        already_created += 1
print(f'Total created consents {count}')
print(f'Already created {already_created}')
print("Expected: ", len(s_consent_data))
print("Existing: ", SubjectConsent.objects.all().count())

Total created consents 906
Already created 0
Expected:  906
Existing:  906


In [5]:
# Registered Subject
from edc_registration.models import RegisteredSubject

fname = path + 'registered_subject.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

rs_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        raise ValidationError('Line is not the right size')
    rs_data.append(data)

td = Site.objects.get(id=40)
already_created = 0
count = 0
for rs in rs_data:
    del rs['may_store_samples']
    del rs['hiv_status']
    rs.update(site=td)
    del rs['survival_status']
    del rs['_state']
    del rs['max_subjects']
    del rs['study_site']
    del rs['subject_types']
    rs.update(
        created=make_aware(parser.parse(rs.get('created'))),
        id=uuid.UUID(rs.get('id'))
    )
    if rs['dob']:
        rs.update(dob=make_aware(parser.parse(rs.get('dob'))).date())
    else:
        rs.update(dob=None)
    if rs['modified']:
        rs.update(modified=make_aware(parser.parse(rs.get('modified'))))
    else:
        rs.update(modified=None)
    if rs['screening_datetime']:
        rs.update(
            screening_datetime=make_aware(parser.parse(rs.get('screening_datetime'))))
    else:
        rs.update(screening_datetime=None)
    if rs['registration_datetime']:
        rs.update(
            registration_datetime=make_aware(parser.parse(rs.get('registration_datetime'))))
    else:
        rs.update(registration_datetime=None)
    if rs['randomization_datetime']:
        rs.update(
            randomization_datetime=make_aware(parser.parse(rs.get('randomization_datetime'))))
    else:
        rs.update(randomization_datetime=None)
    if rs['screening_age_in_years']:
        rs.update(screening_age_in_years=int(float(rs['screening_age_in_years'])))
    else:
        rs.update(screening_age_in_years=None)
    if not rs['subject_identifier'][-3:] == '-10' and not len(rs['subject_identifier']) == 36:
        try:
            subject_screening = SubjectScreening.objects.get(subject_identifier=rs['subject_identifier'])
        except SubjectScreening.DoesNotExist:
            raise ValidationError('Subject screening can not be missing', rs['subject_identifier'])
        else:
            rs.update(screening_identifier=subject_screening.screening_identifier)
    try:
        RegisteredSubject.objects.get(subject_identifier=rs.get('subject_identifier'))
    except RegisteredSubject.DoesNotExist:
        registered_subject = RegisteredSubject(**rs)
        registered_subject.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print('Total created registered subjects: ', count)
print('Already created: ', already_created)
print("Expected: ", len(rs_data))
print("Existing: ", RegisteredSubject.objects.all().count())

Total created registered subjects:  1022
Already created:  0
Expected:  1022
Existing:  1022


In [6]:
# Antenatal Enrollment
import uuid
from td_maternal.models import AntenatalEnrollment
from django.core.exceptions import ValidationError

fname = path + 'antenatal_enrollment.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

antenatal_enrollment_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    antenatal_enrollment_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
skip_list = ['085-40990560-4', '085-40990487-1', '085-40990402-0']
already_created = 0
for ae in antenatal_enrollment_data:
    del ae['_state']
    created = parser.parse(ae.get('created'))
    created = make_aware(created)
    ae.update(
        created=created,
        report_datetime=make_aware(parser.parse(ae.get('report_datetime'))),
        id=uuid.UUID(ae.get('id'))
    )
    if ae['modified']:
        modified = parser.parse(ae.get('modified'))
        modified = make_aware(modified)
        ae.update(modified=modified)
    else:
        ae.update(modified=None)
    del ae['consent_version']
    del ae['export_change_type']
    del ae['export_uuid']
    del ae['exported']
    del ae['exported_datetime']
    del ae['registered_subject_id']
    del ae['_registered_subject_cache']
    if ae['date_at_32wks']:
        ae.update(date_at_32wks=make_aware(parser.parse(ae.get('date_at_32wks'))).date())
    else:
        ae.update(date_at_32wks=None)
    if ae['edd_by_lmp']:
        ae.update(edd_by_lmp=make_aware(parser.parse(ae.get('edd_by_lmp'))).date())
    else:
        ae.update(edd_by_lmp=None)
    if ae['last_period_date']:
        ae.update(last_period_date=make_aware(parser.parse(ae.get('last_period_date'))).date())
    else:
        ae.update(last_period_date=None)
    if ae['rapid_test_date']:
        ae.update(rapid_test_date=make_aware(parser.parse(ae.get('rapid_test_date'))).date())
    else:
        ae.update(rapid_test_date=None)
    if ae['week32_test_date']:
        ae.update(week32_test_date=make_aware(parser.parse(ae.get('week32_test_date'))).date())
    else:
        ae.update(week32_test_date=None)
    if ae['ga_lmp_anc_wks']:
        ae.update(ga_lmp_anc_wks=int(float(ae['ga_lmp_anc_wks'])))
    else:
        ae.update(ga_lmp_anc_wks=None)
    if ae['ga_lmp_enrollment_wks']:
        ae.update(ga_lmp_enrollment_wks=int(float(ae['ga_lmp_enrollment_wks'])))
    else:
        ae.update(ga_lmp_enrollment_wks=None)
    try:
        AntenatalEnrollment.objects.get(subject_identifier=ae.get('subject_identifier'))
    except AntenatalEnrollment.DoesNotExist:
#         if not ae.get('subject_identifier') in skip_list:
        antenatal_enrollment = AntenatalEnrollment(**ae)
        antenatal_enrollment.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created antenatal_enrollment {count}')
print(f'Already created antenatal_enrollment {already_created}')
print("Expected: ", len(antenatal_enrollment_data))
print("Existing: ", AntenatalEnrollment.objects.all().count())

Total created antenatal_enrollment 555
Already created antenatal_enrollment 0
Expected:  555
Existing:  555


In [7]:
# Antenatal Visit Membership
import uuid
from td_maternal.models import AntenatalVisitMembership
from django.core.exceptions import ValidationError

fname = path + 'antenatal_visit_membership.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

antenatal_visit_membership_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    antenatal_visit_membership_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for avm in antenatal_visit_membership_data:
    del avm['_state']
    created = parser.parse(avm.get('created'))
    created = make_aware(created)
    avm.update(
        created=created,
        report_datetime=make_aware(parser.parse(avm.get('report_datetime'))),
        id=uuid.UUID(avm.get('id'))
    )
    if avm['modified']:
        modified = parser.parse(avm.get('modified'))
        modified = make_aware(modified)
        avm.update(modified=modified)
    else:
        avm.update(modified=None)
    del avm['consent_version']
    del avm['export_change_type']
    del avm['export_uuid']
    del avm['exported']
    del avm['exported_datetime']
    del avm['registered_subject_id']
    del avm['_registered_subject_cache']
    try:
        AntenatalVisitMembership.objects.get(subject_identifier=avm.get('subject_identifier'))
    except AntenatalVisitMembership.DoesNotExist:
        antenatal_visit_membership = AntenatalVisitMembership(**avm)
        antenatal_visit_membership.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created antenatal_visit_membership {count}')
print(f'Already created antenatal_visit_membership {already_created}')
print("Expected: ", len(antenatal_visit_membership_data))
print("Existing: ", AntenatalVisitMembership.objects.all().count())

Total created antenatal_visit_membership 510
Already created antenatal_visit_membership 0
Expected:  510
Existing:  510


In [8]:
# Specimen Consent
import uuid
from td_maternal.models import SpecimenConsent
from django.core.exceptions import ValidationError

fname = path + 'specimen_consent.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

specimen_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    specimen_consent_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for sc in specimen_consent_data:
    del sc['_state']
    created = parser.parse(sc.get('created'))
    created = make_aware(created)
    sc.update(
        created=created,
        id=uuid.UUID(sc.get('id'))
    )
    if sc['consent_datetime']:
        consent_datetime = parser.parse(sc.get('consent_datetime'))
        consent_datetime = make_aware(consent_datetime)
        sc.update(consent_datetime=consent_datetime)
    else:
        sc.update(consent_datetime=None)
    if sc['modified']:
        modified = parser.parse(sc.get('modified'))
        modified = make_aware(modified)
        sc.update(modified=modified)
    else:
        sc.update(modified=None)
    if sc['is_verified_datetime']:
        is_verified_datetime = parser.parse(sc.get('is_verified_datetime'))
        is_verified_datetime = make_aware(is_verified_datetime)
        sc.update(
        is_verified_datetime=is_verified_datetime)
    else:
        sc.update(is_verified_datetime=None)
    if sc['is_verified'] == 'TRUE':
        sc.update(is_verified=True)
    elif sc['is_verified'] == 'FALSE':
        sc.update(is_verified=False)
    if sc['offered_copy']:
        sc.update(consent_copy=sc['offered_copy'])
    else:
        sc.update(consent_copy=None)
    del sc['offered_copy']
    if sc['purpose_explained']:
        sc.update(consent_reviewed=sc['purpose_explained'])
    else:
        sc.update(consent_reviewed=None)
    if sc['purpose_understood']:
        sc.update(assessment_score=sc['purpose_understood'])
    else:
        sc.update(assessment_score=None)
    del sc['purpose_understood']
    del sc['purpose_explained']
    del sc['version']
    del sc['export_change_type']
    del sc['export_uuid']
    del sc['exported']
    del sc['exported_datetime']
    del sc['registered_subject_id']
    del sc['_registered_subject_cache']
    try:
        SpecimenConsent.objects.get(subject_identifier=sc.get('subject_identifier'))
    except SpecimenConsent.DoesNotExist:
        specimen_consent = SpecimenConsent(**sc)
        specimen_consent.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created specimen_consent {count}')
print(f'Already created specimen_consent {already_created}')
print("Expected: ", len(specimen_consent_data))
print("Existing: ", SpecimenConsent.objects.all().count())

Total created specimen_consent 555
Already created specimen_consent 0
Expected:  555
Existing:  555


In [9]:
for rs in RegisteredSubject.objects.all():
    sc = SubjectConsent.objects.filter(subject_identifier=rs.subject_identifier).order_by('-created').last()
    if sc and not rs.consent_datetime:
        rs.consent_datetime=sc.consent_datetime
        rs.save_base(raw=True)

In [10]:
# Import Locators
from td_maternal.models import MaternalLocator, SubjectConsent
from edc_action_item.models import ActionItem

fname = path + 'maternal_locator.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

sl_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    sl_data.append(data)
    count += 1

l = [
    '085-40990064-5-10',
    '085-40990196-4-10',
    '085-40990209-3-10',
    '085-40990250-2-10',
    '085-40990223-3-10',
    '085-40990274-5-10'
]
td = Site.objects.get(id=40)
already_created = 0
count = 0
infant_locators = []
new_ids = []
for sl in sl_data:
    del sl['_registered_subject_cache']
    del sl['registered_subject_id']
    del sl['_state']
    del sl['export_change_type']
    del sl['export_uuid']
    del sl['exported']
    del sl['exported_datetime']
    sl.update(site=td)
    sl.update(
        created=make_aware(parser.parse(sl.get('created'))),
        id=uuid.UUID(sl.get('id'))
    )
    if sl['may_sms_follow_up']:
        sl.update(may_sms=sl.get('may_sms_follow_up'))
        del sl['may_sms_follow_up']
    else:
        del sl['may_sms_follow_up']
    if sl['may_contact_someone']:
        sl.update(may_contact_indirectly=sl.get('may_contact_someone'))
        del sl['may_contact_someone']
    else:
        del sl['may_contact_someone']
    if sl['contact_name']:
        sl.update(indirect_contact_name=sl.get('contact_name'))
        del sl['contact_name']
    else:
        del sl['contact_name']
    if sl['contact_rel']:
        sl.update(indirect_contact_relation=sl.get('contact_rel'))
        del sl['contact_rel']
    else:
        del sl['contact_rel']
    if sl['contact_physical_address']:
        sl.update(indirect_contact_physical_address=sl.get('contact_physical_address'))
        del sl['contact_physical_address']
    else:
        del sl['contact_physical_address']
    if sl['contact_cell']:
        sl.update(indirect_contact_cell=sl.get('contact_cell'))
        del sl['contact_cell']
    else:
        del sl['contact_cell']
    if sl['contact_phone']:
        sl.update(indirect_contact_phone=sl.get('contact_phone'))
        del sl['contact_phone']
    else:
        del sl['contact_phone']
    if sl['subject_work_phone']:
        sl.update(subject_work_cell=sl.get('subject_work_phone'))
        del sl['subject_work_phone']
    else:
        del sl['subject_work_phone']
    if sl['care_clinic']:
        sl.update(health_care_infant=sl.get('care_clinic'))
        del sl['care_clinic']
    else:
        del sl['care_clinic']
    if sl['may_follow_up']:
        sl.update(may_call=sl.get('may_follow_up'))
        del sl['may_follow_up']
    else:
        del sl['may_follow_up']
    if sl['home_visit_permission']:
        sl.update(may_visit_home=sl.get('home_visit_permission'))
        del sl['home_visit_permission']
    else:
        del sl['home_visit_permission']
    if sl['modified']:
        sl.update(modified=make_aware(parser.parse(sl.get('modified'))))
    else:
        sl.update(modified=None)
    if sl['report_datetime']:
        sl.update(report_datetime=make_aware(parser.parse(sl.get('report_datetime'))))
    else:
        sl.update(report_datetime=None)
    if sl['date_signed']:
        sl.update(locator_date=make_aware(parser.parse(sl.get('date_signed'))))
        del sl['date_signed']
    else:
        sl.update(locator_date=None)
        del sl['date_signed']
    if sl.get('subject_identifier')[-3:] == '-10':
        infant_locators.append(sl.get('subject_identifier'))
        try:
            rs = RegisteredSubject.objects.get(subject_identifier=sl.get('subject_identifier'))
        except RegisteredSubject.DoesNotExist:
            print(sl.get('subject_identifier'), 'missing rs')
        else:
            sl.update(subject_identifier=rs.relative_identifier)
    try:
        MaternalLocator.objects.get(subject_identifier=sl.get('subject_identifier'))
    except MaternalLocator.DoesNotExist:
        subject_locator = MaternalLocator(**sl)
        subject_locator.save()
        count += 1
    else:
        already_created += 1
print(f'Total created subject locator {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sl_data))
print("Existing: ", MaternalLocator.objects.all().count())

Total created subject locator 546
Already created 4
Expected:  550
Existing:  546


In [11]:
print(len(infant_locators))
infant_locators

6


['085-40990064-5-10',
 '085-40990196-4-10',
 '085-40990209-3-10',
 '085-40990250-2-10',
 '085-40990223-3-10',
 '085-40990274-5-10']

In [12]:
# Import appointments
from edc_appointment.models import Appointment

fname = path + 'appointments.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

schedule_n_visit_schedule = [
    ['1000M', 'V3', 'antenatal_schedule_3', 'anv_schedule_v3'],
    ['1000M', 'V1', 'antenatal_schedule_1', 'anv_schedule_v1'],
    ['1010M', 'V1', 'anv_membership_v1', 'anv_membership_v1'],
    ['1010M', 'V3', 'anv_membership_v3', 'anv_membership_v3'],
    ['1020M', 'V3', 'anv_membership_v3', 'anv_membership_v3'],
    ['1020M', 'V1', 'anv_membership_v1', 'anv_membership_v1'],
    ['2000M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2000M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2010M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2010M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2020M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2020M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2060M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2060M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2120M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2120M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2180M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2180M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2240M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2240M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2300M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3'],
    ['2300M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2360M', 'V1', 'mld_schedule_1', 'mtl_visit_schedule_v1'],
    ['2360M', 'V3', 'mld_schedule_3', 'mtl_visit_schedule_v3']]

ap_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    ap_data.append(data)
site = Site.objects.all()[0]
count = 0
already_created = 0
for ap in ap_data:
    if not sl.get('subject_identifier')[-3:] == '-10':
        ap.update(
            created=make_aware(parser.parse(ap.get('created'))),
            id=uuid.UUID(ap.get('id')),
            facility_name='5-day clinic',
        )
        instruction = ap.get('instruction')
        for code, instruction, schedule, visit_schedule in schedule_n_visit_schedule:
            if code == ap.get('visit_code') and instruction == ap.get('instruction'):
                ap.update(
                    schedule_name=schedule,
                    visit_schedule_name=visit_schedule
                )
        ap.update(timepoint=float(ap['timepoint']))
        if ap['modified']:
            ap.update(modified=make_aware(parser.parse(ap.get('modified'))))
        else:
            ap.update(modified=None)
        if ap['appt_status']:
            ap.update(appt_status=ap.get('appt_status'))
        else:
            ap.update(appt_status=None)
        if ap['appt_datetime']:
            ap.update(
            appt_datetime=make_aware(parser.parse(ap.get('appt_datetime'))))
        else:
            ap.update(appt_datetime=None)
        
        if ap['timepoint_datetime']:
            ap.update(
            timepoint_datetime=make_aware(parser.parse(ap.get('timepoint_datetime'))))
        else:
            ap.update(timepoint_datetime=None)
        
        if ap['appt_close_datetime']:
            ap.update(
            appt_close_datetime=make_aware(parser.parse(ap.get('appt_close_datetime'))))
        else:
            ap.update(appt_close_datetime=None)
        ap.update(site=Site.objects.get(id=40))
        if ap['is_confirmed'] == 'False':
            ap.update(is_confirmed=False)
        elif ap['is_confirmed'] == 'True':
            ap.update(is_confirmed=True)
        if ap['visit_instance']:
            ap.update(visit_code_sequence=int(ap['visit_instance']))
        else:
            ap.update(visit_code_sequence=0)
        del ap['contact_tel']
        del ap['contact_count']
        del ap['visit_instance']
        del ap['_registered_subject_cache']
        del ap['_state']
        del ap['dashboard_type']
        del ap['_time_point_status_cache']
        del ap['_visit_definition_cache']
        del ap['visit_definition_id']
        del ap['registered_subject_id']
        del ap['best_appt_datetime']
        del ap['instruction']
        del ap['time_point_status_id']
        try:
            Appointment.objects.get(id=ap.get('id'))
        except Appointment.DoesNotExist:
            appointment = Appointment(**ap)
            appointment.save_base(raw=True)
            count += 1
        else:
            already_created += 1
    
print(f'Total created subject appointment {count}')
print(f'Already created {already_created}')
print("Expected: ", len(ap_data))
print("Existing: ", Appointment.objects.all().count())

Total created subject appointment 9735
Already created 0
Expected:  9735
Existing:  9735


In [13]:
# Maternal visit
from datetime import timedelta
from td_maternal.models import MaternalVisit
from edc_appointment.models import Appointment
from django.db.utils import IntegrityError

fname = path + 'maternal_visits.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


sv_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    sv_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for sv in sv_data:
    if not sv.get('subject_identifier')[-3:] == '-10':
        del sv['_state']
        del sv['_appointment_cache']
        sv.update(
            created=make_aware(parser.parse(sv.get('created'))),
            id=uuid.UUID(sv.get('id')),
        )
        if sv['report_datetime']:
            sv.update(
                report_datetime=make_aware(parser.parse(sv.get('report_datetime'))))
        else:
            sv.update(report_datetime=None)
        if sv['modified']:
            sv.update(
                modified=make_aware(parser.parse(sv.get('modified'))))
        else:
            sv.update(modified=None)
        if sv['last_alive_date']:
            sv.update(
                last_alive_date=make_aware(parser.parse(sv.get('last_alive_date'))).date())
        else:
            sv.update(last_alive_date=None)
        try:
            appointment = Appointment.objects.get(id=sv.get('appointment_id'))
        except Appointment.DoesNotExist:
            print(sv.get('appointment_id'))
            count_missing += 1
        else:
            sv.update(
                appointment=appointment,
                schedule_name=appointment.schedule_name,
                visit_schedule_name=appointment.visit_schedule_name,
                site=appointment.site,
                visit_code=appointment.visit_code
            )
        del sv['export_change_type']
        del sv['export_uuid']
        del sv['exported']
        del sv['exported_datetime']
        try:
            MaternalVisit.objects.get(id=sv.get('id'))
        except MaternalVisit.DoesNotExist:
            maternal_visit = MaternalVisit(**sv)
            maternal_visit.save_base(raw=True)
            count += 1
        else:
            already_created += 1
print(f'Total created subject visits {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sv_data))
print("Existing: ", MaternalVisit.objects.all().count())

Total created subject visits 4069
Already created 0
Expected:  4069
Existing:  4069


In [14]:
# Maternal Labour Delivery
from datetime import timedelta
from td_maternal.models import MaternalLabourDel
from django.db.utils import IntegrityError

fname = path + 'maternal_labour_del.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


mld_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    mld_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for mld in mld_data:
    del mld['_state']
    mld.update(
        created=make_aware(parser.parse(mld.get('created'))),
        id=uuid.UUID(mld.get('id')),
    )
    if mld['report_datetime']:
        mld.update(
            report_datetime=make_aware(parser.parse(mld.get('report_datetime'))))
    else:
        mld.update(report_datetime=None)
    if mld['arv_initiation_date']:
        mld.update(
            arv_initiation_date=make_aware(parser.parse(mld.get('arv_initiation_date'))).date())
    else:
        mld.update(arv_initiation_date=None)
    if mld['modified']:
        mld.update(
            modified=make_aware(parser.parse(mld.get('modified'))))
    else:
        mld.update(modified=None)
    if mld['delivery_datetime']:
        mld.update(
            delivery_datetime=make_aware(parser.parse(mld.get('delivery_datetime'))))
    else:
        mld.update(delivery_datetime=None)
    del mld['export_change_type']
    del mld['export_uuid']
    del mld['exported']
    del mld['exported_datetime']
    del mld['consent_version']
    del mld['registered_subject_id']
    del mld['_registered_subject_cache']
    try:
        MaternalLabourDel.objects.get(id=mld.get('id'))
    except MaternalLabourDel.DoesNotExist:
        maternal_labour_del = MaternalLabourDel(**mld)
        maternal_labour_del.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created maternal labour delivery {count}')
print(f'Already created {already_created}')
print("Expected: ", len(mld_data))
print("Existing: ", MaternalLabourDel.objects.all().count())

Total created maternal labour delivery 468
Already created 0
Expected:  468
Existing:  468


In [15]:
# Import CRFs
from pprint import pprint
import csv
from decimal import Decimal
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
crf_list = [
    'maternalultrasoundinitial',
    'maternalobstericalhistory',
    'maternalmedicalhistory',
    'maternaldemographics',
    'maternallifetimearvhistory',
    'maternalarvpreg',
    'maternalclinicalmeasurementsone',
    'maternalrando',
    'maternalinterimidcc',
    'maternalclinicalmeasurementstwo',
    'rapidtestresult',
    'maternaldiagnoses',
    'maternalsubstanceusepriorpreg',
    'maternalpostpartumdep',
    'maternalpostpartumfu',
    'maternalarvpost',
    'maternalcontraception',
    'maternalsrh',
    'rapidtestresult',
    'maternalsubstanceuseduringpreg',
    'maternalhivinterimhx',
    'maternalarvpostadh'
    ]

data_path = path + 'maternal_crfs/'

missing_facility = []
for crf_name in crf_list:
    fname = data_path + crf_name + '.csv'
    with open(fname, encoding="ISO-8859-1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        header = next(csv_reader)
        del header[0]
        created = 0
        already_exists = 0
        count = 0
        for line in csv_reader:
            del line[0]
            if len(line) == len(header):
                data = dict(zip(header, line))
            else:
                print(lines[count - 1])
                print(line)
                print(lines[count + 1])
                raise ValidationError('Line is not the right size')
            td = Site.objects.get(id=40)
            data.update(
                site=td,
                created=make_aware(parser.parse(data.get('created'))),
                id=uuid.UUID(data.get('id'))
            )
            if data['modified']:
                data.update(
                    modified=make_aware(parser.parse(data.get('modified'))))
            else:
                data.update(modified=None)
            if data['report_datetime']:
                data.update(
                    report_datetime=make_aware(parser.parse(data.get('report_datetime'))))
            else:
                data.update(report_datetime=None)
            del data['_state']
            del data['_maternal_visit_cache']
            del data['export_change_type']
            del data['export_uuid']
            del data['exported']
            del data['exported_datetime']
            crf_cls = django_apps.get_model('td_maternal', crf_name)
            if 'rapidtestresult' == crf_name:
                if data['result_date']:
                    data.update(
                        result_date=make_aware(parser.parse(data.get('result_date'))).date())
                else:
                    data.update(result_date=None)
            if 'maternalcontraception' == crf_name:
                if data['date_notified']:
                    data.update(
                        date_notified=make_aware(parser.parse(data.get('date_notified'))).date())
                else:
                    data.update(date_notified=None)
                if data['pap_smear_date']:
                    data.update(
                        pap_smear_date=make_aware(parser.parse(data.get('pap_smear_date'))).date())
                else:
                    data.update(pap_smear_date=None)
                if data['pregnancy_date']:
                    data.update(
                        pregnancy_date=make_aware(parser.parse(data.get('pregnancy_date'))).date())
                else:
                    data.update(pregnancy_date=None)
                if data['contraceptive_startdate']:
                    data.update(
                        contraceptive_startdate=make_aware(parser.parse(data.get('contraceptive_startdate'))).date())
                else:
                    data.update(contraceptive_startdate=None)
            if 'maternalinterimidcc' == crf_name:
                if data['recent_cd4_date']:
                    data.update(
                        recent_cd4_date=make_aware(parser.parse(data.get('recent_cd4_date'))).date())
                else:
                    data.update(recent_cd4_date=None)
                if data['recent_vl_date']:
                    data.update(
                        recent_vl_date=make_aware(parser.parse(data.get('recent_vl_date'))).date())
                else:
                    data.update(recent_vl_date=None)
                if data['recent_cd4']:
                    data.update(
                        recent_cd4=int(float(data.get('recent_cd4'))))
                else:
                    data.update(recent_cd4=None)
                if data['value_vl']:
                    data.update(
                        value_vl=int(float(data.get('value_vl'))))
                else:
                    data.update(value_vl=None)
            if 'maternallifetimearvhistory' == crf_name:
                if data['haart_start_date']:
                    data.update(
                        haart_start_date=make_aware(parser.parse(data.get('haart_start_date'))).date())
                else:
                    data.update(haart_start_date=None)
            if 'maternalmedicalhistory' == crf_name:
                if data['date_hiv_diagnosis']:
                    data.update(
                        date_hiv_diagnosis=make_aware(parser.parse(data.get('date_hiv_diagnosis'))).date())
                else:
                    data.update(date_hiv_diagnosis=None)
                if data['cd4_date']:
                    data.update(
                        cd4_date=make_aware(parser.parse(data.get('cd4_date'))).date())
                else:
                    data.update(cd4_date=None)
                if data['cd4_count']:
                    data.update(
                        cd4_count=int(float(data.get('cd4_count'))))
                else:
                    data.update(cd4_count=None)  
            if 'maternalultrasoundinitial' == crf_name:
                if data['edd_confirmed']:
                    data.update(
                        edd_confirmed=make_aware(parser.parse(data.get('edd_confirmed'))).date())
                else:
                    data.update(edd_confirmed=None)
                if data['ac']:
                    data.update(
                        ac=float(data.get('ac')))
                else:
                    data.update(ac=None)
                if data['hc']:
                    data.update(
                        hc=float(data.get('hc')))
                else:
                    data.update(hc=None)
                if data['bpd']:
                    data.update(
                        bpd=float(data.get('bpd')))
                else:
                    data.update(bpd=None)
                if data['fl']:
                    data.update(
                        fl=float(data.get('fl')))
                else:
                    data.update(fl=None)
                if data['ga_by_ultrasound_days']:
                    data.update(
                        ga_by_ultrasound_days=int(float(data.get('ga_by_ultrasound_days'))))
                else:
                    data.update(ga_by_ultrasound_days=None)
                if data['ga_by_ultrasound_wks']:
                    data.update(
                        ga_by_ultrasound_wks=int(float(data.get('ga_by_ultrasound_wks'))))
                else:
                    data.update(ga_by_ultrasound_wks=None)
                if data['ga_by_lmp']:
                    data.update(
                        ga_by_lmp=int(float(data.get('ga_by_lmp'))))
                else:
                    data.update(ga_by_lmp=None)
                if data['ga_confirmed']:
                    data.update(
                        ga_confirmed=int(float(data.get('ga_confirmed'))))
                else:
                    data.update(ga_confirmed=None) 
                if data['ga_confirmed']:
                    data.update(
                        ga_confirmed=int(float(data.get('ga_confirmed'))))
                else:
                    data.update(ga_confirmed=None)  
                if data['est_fetal_weight']:
                    data.update(
                        est_fetal_weight=float(data.get('est_fetal_weight')))
                else:
                    data.update(est_fetal_weight=None)
                if data['est_edd_ultrasound']:
                    data.update(
                        est_edd_ultrasound=make_aware(parser.parse(data.get('est_edd_ultrasound'))).date())
                else:
                    data.update(est_edd_ultrasound=None)
            if 'maternalrando' == crf_name:
                if data['randomization_datetime']:
                    data.update(
                        randomization_datetime=make_aware(parser.parse(data.get('randomization_datetime'))))
                else:
                    data.update(randomization_datetime=None)
                if data['sid']:
                    data.update(sid=int(data['sid']))
                del data['subject_identifier']
            if 'maternalpostpartumfu' == crf_name:
                if data['hospitalization_days']:
                    data.update(
                        hospitalization_days=int(float(data.get('hospitalization_days'))))
                else:
                    data.update(hospitalization_days=None)
            if 'maternalhivinterimhx' == crf_name:
                if data['cd4_date']:
                    data.update(
                        cd4_date=make_aware(parser.parse(data.get('cd4_date'))).date())
                else:
                    data.update(cd4_date=None)
                if data['vl_date']:
                    data.update(
                        vl_date=make_aware(parser.parse(data.get('vl_date'))).date())
                else:
                    data.update(vl_date=None)
            if 'maternalarvpostadh' == crf_name:
                if data['missed_doses']:
                    data.update(missed_doses=int(float(data['missed_doses'])))
                else:
                    data.update(missed_doses=0)
                if data['missed_days']:
                    data.update(missed_days=int(float(data['missed_days'])))
                else:
                    data.update(missed_days=0)
                if data['missed_days_discnt']:
                    data.update(missed_days_discnt=int(float(data['missed_days_discnt'])))
                else:
                    data.update(missed_days_discnt=0)
            try:
                maternal_visit = MaternalVisit.objects.get(id=data['maternal_visit_id'])
            except MaternalVisit.DoesNotExist:
                raise ValidationError('Visit can not be missing')
            else:
                data.update(maternal_visit=maternal_visit)
                del data['maternal_visit_id']
            try:
                crf_cls.objects.get(id=data.get('id'))
            except crf_cls.DoesNotExist:
                try:
                    obj = crf_cls(**data)
                    obj.save_base(raw=True)
                    created += 1
                except Exception as e:
                    print(e)
            else:
                already_exists += 1
        print('**************************************************')
        print(f'Created: {created} for {crf_name}')
        print(f'Already Exists: {already_exists} for {crf_name}')
        print('###################################################')
print('Done woth all CRFS')

**************************************************
Created: 500 for maternalultrasoundinitial
Already Exists: 0 for maternalultrasoundinitial
###################################################
**************************************************
Created: 484 for maternalobstericalhistory
Already Exists: 0 for maternalobstericalhistory
###################################################
**************************************************
Created: 493 for maternalmedicalhistory
Already Exists: 0 for maternalmedicalhistory
###################################################
**************************************************
Created: 494 for maternaldemographics
Already Exists: 0 for maternaldemographics
###################################################
**************************************************
Created: 322 for maternallifetimearvhistory
Already Exists: 0 for maternallifetimearvhistory
###################################################
********************************************

In [16]:
# Maternal Labour Delivery
from datetime import timedelta
from td_maternal.models import MaternalRequisition
from edc_lab.models import Panel
from django.db.utils import IntegrityError

fname = path + 'maternal_crfs/maternal_requisition.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


rm_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    rm_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for mr in rm_data:
    del mr['_state']
    mr.update(
        created=make_aware(parser.parse(mr.get('created'))),
        id=uuid.UUID(mr.get('id')),
        protocol_number='085'
    )
    if mr['report_datetime']:
        mr.update(
            report_datetime=make_aware(parser.parse(mr.get('report_datetime'))))
    else:
        mr.update(report_datetime=None)
    if mr['requisition_datetime']:
        mr.update(
            requisition_datetime=make_aware(parser.parse(mr.get('requisition_datetime'))))
    else:
        mr.update(requisition_datetime=None)
    if mr['modified']:
        mr.update(
            modified=make_aware(parser.parse(mr.get('modified'))))
    else:
        mr.update(modified=None)
    if mr['is_receive_datetime']:
        mr.update(
            received_datetime=make_aware(parser.parse(mr.get('is_receive_datetime'))))
    else:
        mr.update(received_datetime=None)
    if mr['drawn_datetime']:
        mr.update(
            drawn_datetime=make_aware(parser.parse(mr.get('drawn_datetime'))))
    else:
        mr.update(drawn_datetime=None)
    if mr['estimated_volume']:
        mr.update(
            estimated_volume=float(mr.get('estimated_volume')))
    else:
        mr.update(estimated_volume=None)
    if mr['packed'] == 'False':
        mr.update(packed=False)
    elif mr['packed'] == 'True':
        mr.update(packed=True)
    else:
        mr.update(packed=None)
    if mr['received'] == 'False':
        mr.update(received=False)
    elif mr['received'] == 'True':
        mr.update(received=True)
    else:
        mr.update(received=None)
    try:
        maternal_visit = MaternalVisit.objects.get(id=mr['maternal_visit_id'])
    except MaternalVisit.DoesNotExist:
        raise ValidationError('Visit can not be missing')
    else:
        mr.update(maternal_visit=maternal_visit)
        del mr['maternal_visit_id']
    del mr['export_change_type']
    del mr['is_receive_datetime']
    del mr['export_uuid']
    del mr['exported']
    del mr['exported_datetime']
    del mr['_maternal_visit_cache']
    del mr['is_packed']
    del mr['is_receive']
    del mr['aliquot_type_id']
    del mr['is_labelled_datetime']
    del mr['study_site']
    del mr['panel_id']
    del mr['protocol']
    del mr['packing_list_id']
    del mr['is_lis']
    del mr['item_count_total']
    del mr['is_labelled']
    del mr['_panel_cache']
    del mr['specimen_identifier']
    if mr['panel_name'] == 'Viral Load':
        mr.update(panel_name='viral_load')
    elif mr['panel_name'] == 'Glucose 2h':
        mr.update(panel_name='glucose_2h')
    elif mr['panel_name'] == 'Glucose 1h':
        mr.update(panel_name='glucose_1h')
    elif mr['panel_name'] == 'Fasting Glucose':
        mr.update(panel_name='fasting_glucose')
    elif mr['panel_name'] == 'PBMC VL':
        mr.update(panel_name='pbmc_vl')
    elif mr['panel_name'] == 'PBMC Plasma (STORE ONLY)':
        mr.update(panel_name='pbmc_pl_store')
    try:
        panel = Panel.objects.get(name=mr['panel_name'])
    except Panel.DoesNotExist:
        raise ValidationError('Panel can not be missing')
    else:
        mr.update(panel=panel)
        del mr['panel_name']
    try:
        MaternalRequisition.objects.get(id=mr.get('id'))
    except MaternalRequisition.DoesNotExist:
        maternal_requisition = MaternalRequisition(**mr)
        maternal_requisition.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created maternal requisition {count}')
print(f'Already created {already_created}')
print("Expected: ", len(rm_data))
print("Existing: ", MaternalRequisition.objects.all().count())

Total created maternal requisition 3594
Already created 0
Expected:  3594
Existing:  3594


In [17]:
# Infant Birth
import uuid
from td_infant.models import InfantBirth
from django.core.exceptions import ValidationError

fname = path + 'infant_birth.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

infant_birth_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    infant_birth_data.append(data)
    count += 0
    
td = Site.objects.get(id=40)

count = 0
already_created = 0
for ib in infant_birth_data:
    del ib['_state']
    created = parser.parse(ib.get('created'))
    created = make_aware(created)
    ib.update(
        created=created,
        report_datetime=make_aware(parser.parse(ib.get('report_datetime'))),
        id=uuid.UUID(ib.get('id')),
        site=Site.objects.get(id=40)
    )
    if ib['modified']:
        modified = parser.parse(ib.get('modified'))
        modified = make_aware(modified)
        ib.update(modified=modified)
    else:
        ib.update(modified=None)
    if ib['dob']:
        ib.update(dob=make_aware(parser.parse(ib.get('dob'))))
    else:
        ib.update(dob=None)
    del ib['export_change_type']
    del ib['export_uuid']
    del ib['exported']
    del ib['exported_datetime']
    del ib['registered_subject_id']
    del ib['_registered_subject_cache']
    del ib['maternal_labour_del_id']
    try:
        InfantBirth.objects.get(subject_identifier=ib.get('subject_identifier'))
    except InfantBirth.DoesNotExist:
        infant_birth = InfantBirth(**ib)
        infant_birth.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created infant_birth {count}')
print(f'Already created infant_birth {already_created}')
print("Expected: ", len(infant_birth_data))
print("Existing: ", InfantBirth.objects.all().count())

Total created infant_birth 462
Already created infant_birth 0
Expected:  462
Existing:  462


In [18]:
# Import appointments
from td_infant.models import Appointment

fname = path + 'infant_appointments.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]

schedule_n_visit_schedule = [
    ['2000', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2000', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2010', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2010', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2020', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2020', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2060', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2060', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2120', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2120', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2180', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2180', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2240', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2240', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2300', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3'],
    ['2300', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2360', 'V1', 'infant_schedule_v1', 'infant_visit_schedule_v1'],
    ['2360', 'V3', 'infant_schedule_v3', 'infant_visit_schedule_v3']
]

ap_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    ap_data.append(data)
site = Site.objects.all()[0]
count = 0
already_created = 0
for ap in ap_data:
    ap.update(
        created=make_aware(parser.parse(ap.get('created'))),
        id=uuid.UUID(ap.get('id')),
        facility_name='5-day clinic',
    )
    instruction = ap.get('instruction')
    for code, instruction, schedule, visit_schedule in schedule_n_visit_schedule:
        if code == ap.get('visit_code') and instruction == ap.get('instruction'):
            ap.update(
                schedule_name=schedule,
                visit_schedule_name=visit_schedule
            )
    ap.update(timepoint=float(ap['timepoint']))
    if ap['modified']:
        ap.update(modified=make_aware(parser.parse(ap.get('modified'))))
    else:
        ap.update(modified=None)
    if ap['appt_datetime']:
        ap.update(
        appt_datetime=make_aware(parser.parse(ap.get('appt_datetime'))))
    else:
        ap.update(appt_datetime=None)

    if ap['timepoint_datetime']:
        ap.update(
        timepoint_datetime=make_aware(parser.parse(ap.get('timepoint_datetime'))))
    else:
        ap.update(timepoint_datetime=None)

    if ap['appt_close_datetime']:
        ap.update(
        appt_close_datetime=make_aware(parser.parse(ap.get('appt_close_datetime'))))
    else:
        ap.update(appt_close_datetime=None)
    ap.update(site=Site.objects.get(id=40))
    if ap['is_confirmed'] == 'False':
        ap.update(is_confirmed=False)
    elif ap['is_confirmed'] == 'True':
        ap.update(is_confirmed=True)
    if ap['visit_instance']:
        ap.update(visit_code_sequence=int(ap['visit_instance']))
    else:
        ap.update(visit_code_sequence=0)
    del ap['contact_tel']
    del ap['contact_count']
    del ap['visit_instance']
    del ap['_registered_subject_cache']
    del ap['_state']
    del ap['dashboard_type']
    del ap['_time_point_status_cache']
    del ap['_visit_definition_cache']
    del ap['visit_definition_id']
    del ap['registered_subject_id']
    del ap['best_appt_datetime']
    del ap['instruction']
    del ap['time_point_status_id']
    try:
        Appointment.objects.get(id=ap.get('id'))
    except Appointment.DoesNotExist:
        appointment = Appointment(**ap)
        appointment.save_base(raw=True)
        count += 1
    else:
        already_created += 1
    
print('Total created infant appointment: ', count)
print(f'Already created {already_created}')
print("Expected: ", len(ap_data))
print("Existing: ", Appointment.objects.all().count())

Total created infant appointment:  4055
Already created 0
Expected:  4055
Existing:  4055


In [19]:
# Subject visit
from datetime import timedelta
from td_infant.models import InfantBirth, Appointment
from django.db.utils import IntegrityError

fname = path + 'infant_visit.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


sv_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    sv_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for sv in sv_data:
    del sv['_state']
    if sv['id'] == '100b9da4-1a8f-4e2c-858f-cd709aa9e3da':
        pprint(sv)
    sv.update(
        created=make_aware(parser.parse(sv.get('created'))),
        id=uuid.UUID(sv.get('id')),
    )
    if sv['report_datetime']:
        sv.update(
            report_datetime=make_aware(parser.parse(sv.get('report_datetime'))))
    else:
        sv.update(report_datetime=None)
    if sv['modified']:
        sv.update(
            modified=make_aware(parser.parse(sv.get('modified'))))
    else:
        sv.update(modified=None)
    if sv['last_alive_date']:
        sv.update(
            last_alive_date=make_aware(parser.parse(sv.get('last_alive_date'))).date())
    else:
        sv.update(last_alive_date=None)
    try:
        appointment = Appointment.objects.get(id=sv.get('appointment_id'))
    except Appointment.DoesNotExist:
        print(sv.get('appointment_id'))
        count_missing += 1
    else:
        sv.update(
            appointment=appointment,
            schedule_name=appointment.schedule_name,
            visit_schedule_name=appointment.visit_schedule_name,
            site=appointment.site,
            visit_code=appointment.visit_code
        )
    del sv['export_change_type']
    del sv['export_uuid']
    del sv['exported']
    del sv['exported_datetime']
    del sv['_appointment_cache']
    try:
        InfantVisit.objects.get(id=sv.get('id'))
    except InfantVisit.DoesNotExist:
        infant_visit = InfantVisit(**sv)
        infant_visit.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created infant visits {count}')
print(f'Already created {already_created}')
print("Expected: ", len(sv_data))
print("Existing: ", InfantVisit.objects.all().count())

{'_appointment_cache': '085-40990310-6-10 infant for 2180.0',
 'appointment_id': '84d835d0-2d59-4368-afad-775948501586',
 'comments': '',
 'created': '2019-07-05 08:40:26.721430',
 'export_change_type': 'I',
 'export_uuid': '2479ac99-6c81-4447-b6fa-9fcb0b4cc8a0',
 'exported': 'False',
 'exported_datetime': '',
 'hostname_created': 'td.bhp.org.bw',
 'hostname_modified': 'td.bhp.org.bw',
 'id': '100b9da4-1a8f-4e2c-858f-cd709aa9e3da',
 'info_source': 'participant',
 'info_source_other': '',
 'information_provider': 'MOTHER',
 'information_provider_other': '',
 'is_present': 'Yes',
 'last_alive_date': '2019-07-05',
 'modified': '2019-07-05 08:40:26.721509',
 'reason': 'scheduled',
 'reason_missed': '',
 'report_datetime': '2019-07-05 08:40:19.000000',
 'require_crfs': 'Yes',
 'revision': '2.1.7:master:43fad92164d8a613937c0a7088003a9d6f2001a5',
 'study_status': 'on study',
 'subject_identifier': '085-40990310-6-10',
 'survival_status': 'alive',
 'user_created': 'gmasasa',
 'user_modified': 

In [20]:
# Import CRFs
from pprint import pprint
import csv
from decimal import Decimal
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
crf_list = [
    'infantbirthdata',
    'infantbirthexam',
    'infantbirthfeedingvaccine',
    'infantbirtharv',
    'infantcongenitalanomalies',
    'infantnvpdispensing',
    'infantfu',
    'infantfuphysical',
    'infantfudx',
    'infantfunewmed',
    'infantarvproph',
    'infantfeeding',
    'solidfoodassessment',
    'infantnvpadjustment',
    'infantfuimmunizations',
]

data_path = path + 'infant_crfs/'

missing_facility = []
for crf_name in crf_list:
    fname = data_path + crf_name + '.csv'
    with open(fname, encoding="ISO-8859-1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        header = next(csv_reader)
        del header[0]
        created = 0
        already_exists = 0
        count = 0
        for line in csv_reader:
            del line[0]
            if len(line) == len(header):
                data = dict(zip(header, line))
            else:
                print(lines[count - 1])
                print(line)
                print(lines[count + 1])
                raise ValidationError('Line is not the right size')
            td = Site.objects.get(id=40)
            data.update(
                site=td,
                created=make_aware(parser.parse(data.get('created'))),
                id=uuid.UUID(data.get('id'))
            )
            if data['modified']:
                data.update(
                    modified=make_aware(parser.parse(data.get('modified'))))
            else:
                data.update(modified=None)
            if data['report_datetime']:
                data.update(
                    report_datetime=make_aware(parser.parse(data.get('report_datetime'))))
            else:
                data.update(report_datetime=None)
            del data['_state']
            del data['_infant_visit_cache']
            del data['export_change_type']
            del data['export_uuid']
            del data['exported']
            del data['exported_datetime']
            crf_cls = django_apps.get_model('td_infant', crf_name)
            
            if 'infantfu' == crf_name:
                if data['days_hospitalized']:
                    data.update(
                        days_hospitalized=int(float(data.get('days_hospitalized'))))
                else:
                    data.update(days_hospitalized=None)
            if 'solidfoodassessment' == crf_name:
                if data['age_solid_food']:
                    data.update(
                        age_solid_food=int(float(data.get('age_solid_food'))))
                else:
                    data.update(age_solid_food=None)
                if data['porridge_freq']:
                    data.update(
                        porridge_freq=int(float(data.get('porridge_freq'))))
                else:
                    data.update(porridge_freq=None)
                if data['tsabana_week']:
                    data.update(
                        tsabana_week=int(float(data.get('tsabana_week'))))
                else:
                    data.update(tsabana_week=None)
                if data['meat_freq']:
                    data.update(
                        meat_freq=int(float(data.get('meat_freq'))))
                else:
                    data.update(meat_freq=None)
                if data['potatoes_freq']:
                    data.update(
                        potatoes_freq=int(float(data.get('potatoes_freq'))))
                else:
                    data.update(potatoes_freq=None)
                if data['carrot_swt_potato_freq']:
                    data.update(
                        carrot_swt_potato_freq=int(float(data.get('carrot_swt_potato_freq'))))
                else:
                    data.update(carrot_swt_potato_freq=None)
                if data['green_veg_freq']:
                    data.update(
                        green_veg_freq=int(float(data.get('green_veg_freq'))))
                else:
                    data.update(green_veg_freq=None)
                if data['fresh_fruits_freq']:
                    data.update(
                        fresh_fruits_freq=int(float(data.get('fresh_fruits_freq'))))
                else:
                    data.update(fresh_fruits_freq=None)
                if data['fullcream_milk_freq']:
                    data.update(
                        fullcream_milk_freq=int(float(data.get('fullcream_milk_freq'))))
                else:
                    data.update(fullcream_milk_freq=None)
                if data['skim_milk_freq']:
                    data.update(
                        skim_milk_freq=int(float(data.get('skim_milk_freq'))))
                else:
                    data.update(skim_milk_freq=None)
                if data['raw_milk_freq']:
                    data.update(
                        raw_milk_freq=int(float(data.get('raw_milk_freq'))))
                else:
                    data.update(raw_milk_freq=None)
                if data['juice_freq']:
                    data.update(
                        juice_freq=int(float(data.get('juice_freq'))))
                else:
                    data.update(juice_freq=None)
                if data['eggs_freq']:
                    data.update(
                        eggs_freq=int(float(data.get('eggs_freq'))))
                else:
                    data.update(eggs_freq=None)
                if data['yogurt_freq']:
                    data.update(
                        yogurt_freq=int(float(data.get('yogurt_freq'))))
                else:
                    data.update(yogurt_freq=None)
                if data['cheese_freq']:
                    data.update(
                        cheese_freq=int(float(data.get('cheese_freq'))))
                else:
                    data.update(cheese_freq=None)
            if 'infantfeeding' == crf_name:
                if data['last_att_sche_visit']:
                    data.update(
                        last_att_sche_visit=make_aware(parser.parse(data.get('last_att_sche_visit'))).date())
                else:
                    data.update(last_att_sche_visit=None)
                if data['formula_intro_date']:
                    data.update(
                        formula_intro_date=make_aware(parser.parse(data.get('formula_intro_date'))).date())
                else:
                    data.update(formula_intro_date=None)
                if data['date_first_formula']:
                    data.update(
                        date_first_formula=make_aware(parser.parse(data.get('date_first_formula'))).date())
                else:
                    data.update(date_first_formula=None)
                if data['most_recent_bm']:
                    data.update(
                        most_recent_bm=make_aware(parser.parse(data.get('most_recent_bm'))).date())
                else:
                    data.update(most_recent_bm=None)
            if 'infantfuphysical' == crf_name:
                if data['weight_kg']:
                    data.update(
                        weight_kg=float(data.get('weight_kg')))
                else:
                    data.update(weight_kg=None)
                if data['height']:
                    data.update(
                        height=float(data.get('height')))
                else:
                    data.update(height=None)
                if data['head_circumference']:
                    data.update(
                        head_circumference=float(data.get('head_circumference')))
                else:
                    data.update(head_circumference=None)
            if 'infantnvpdispensing' == crf_name:
                if data['nvp_admin_date']:
                    data.update(
                        nvp_admin_date=make_aware(parser.parse(data.get('nvp_admin_date'))).date())
                else:
                    data.update(nvp_admin_date=None)
            if 'infantbirtharv' == crf_name:
                if data['azt_dose_date']:
                    data.update(
                        azt_dose_date=make_aware(parser.parse(data.get('azt_dose_date'))).date())
                else:
                    data.update(azt_dose_date=None)
                if data['nvp_dose_date']:
                    data.update(
                        nvp_dose_date=make_aware(parser.parse(data.get('nvp_dose_date'))).date())
                else:
                    data.update(nvp_dose_date=None)
            if 'infantbirthdata' == crf_name:
                if data['apgar_score_min_1']:
                    data.update(
                        apgar_score_min_1=int(float(data.get('apgar_score_min_1'))))
                else:
                    data.update(apgar_score_min_1=None)
                if data['apgar_score_min_5']:
                    data.update(
                        apgar_score_min_5=int(float(data.get('apgar_score_min_5'))))
                else:
                    data.update(apgar_score_min_5=None)
                if data['apgar_score_min_10']:
                    data.update(
                        apgar_score_min_10=int(float(data.get('apgar_score_min_10'))))
                else:
                    data.update(apgar_score_min_10=None)
                if data['infant_length']:
                    data.update(
                        infant_length=float(data.get('infant_length')))
                else:
                    data.update(infant_length=None)
                if data['weight_kg']:
                    data.update(
                        weight_kg=float(data.get('weight_kg')))
                else:
                    data.update(weight_kg=None)
                if data['head_circumference']:
                    data.update(
                        head_circumference=float(data.get('head_circumference')))
                else:
                    data.update(head_circumference=None)
            if 'solidfoodassessment' == crf_name:
                if data['age_solid_food']:
                    data.update(age_solid_food=int(float(data['age_solid_food'])))
                else:
                    data.update(age_solid_food=None)
            try:
                infant_visit = InfantVisit.objects.get(id=data['infant_visit_id'])
            except InfantVisit.DoesNotExist:
                pprint(data)
                raise ValidationError('Visit can not be missing')
            else:
                data.update(infant_visit=infant_visit)
                del data['infant_visit_id']
            try:
                crf_cls.objects.get(id=data.get('id'))
            except crf_cls.DoesNotExist:
                try:
                    obj = crf_cls(**data)
                    obj.save_base(raw=True)
                    created += 1
                except Exception as e:
                    print(e)
            else:
                already_exists += 1
        print('**************************************************')
        print(f'Created: {created} for {crf_name}')
        print(f'Already Exists: {already_exists} for {crf_name}')
        print('###################################################')
print('Done woth all CRFS')

**************************************************
Created: 427 for infantbirthdata
Already Exists: 0 for infantbirthdata
###################################################
**************************************************
Created: 420 for infantbirthexam
Already Exists: 0 for infantbirthexam
###################################################
**************************************************
Created: 426 for infantbirthfeedingvaccine
Already Exists: 0 for infantbirthfeedingvaccine
###################################################
**************************************************
Created: 299 for infantbirtharv
Already Exists: 0 for infantbirtharv
###################################################
**************************************************
Created: 16 for infantcongenitalanomalies
Already Exists: 0 for infantcongenitalanomalies
###################################################
**************************************************
Created: 153 for infantnvpdispensing
Alrea

In [21]:
# Maternal Labour Delivery
from datetime import timedelta
from td_infant.models import InfantRequisition, InfantVisit
from edc_lab.models import Panel
from django.db.utils import IntegrityError

fname = path + 'infant_crfs/infant_requisition.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]


ir_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    data = dict(zip(headers, line))
    ir_data.append(data)

site = Site.objects.all()[0]
count = 0
already_created = 0
for ir in ir_data:
    del ir['_state']
    ir.update(
        created=make_aware(parser.parse(ir.get('created'))),
        id=uuid.UUID(ir.get('id')),
        protocol_number='085'
    )
    if ir['report_datetime']:
        ir.update(
            report_datetime=make_aware(parser.parse(ir.get('report_datetime'))))
    else:
        ir.update(report_datetime=None)
    if ir['requisition_datetime']:
        ir.update(
            requisition_datetime=make_aware(parser.parse(ir.get('requisition_datetime'))))
    else:
        ir.update(requisition_datetime=None)
    if ir['modified']:
        ir.update(
            modified=make_aware(parser.parse(ir.get('modified'))))
    else:
        ir.update(modified=None)
    if ir['is_receive_datetime']:
        ir.update(
            received_datetime=make_aware(parser.parse(ir.get('is_receive_datetime'))))
    else:
        ir.update(received_datetime=None)
    if ir['drawn_datetime']:
        ir.update(
            drawn_datetime=make_aware(parser.parse(ir.get('drawn_datetime'))))
    else:
        ir.update(drawn_datetime=None)
    if ir['estimated_volume']:
        ir.update(
            estimated_volume=float(ir.get('estimated_volume')))
    else:
        ir.update(estimated_volume=None)
    if ir['packed'] == 'False':
        ir.update(packed=False)
    elif ir['packed'] == 'True':
        ir.update(packed=True)
    else:
        ir.update(packed=None)
    if ir['received'] == 'False':
        ir.update(received=False)
    elif ir['received'] == 'True':
        ir.update(received=True)
    else:
        ir.update(received=None)
    try:
        infant_visit = InfantVisit.objects.get(id=ir['infant_visit_id'])
    except InfantVisit.DoesNotExist:
        raise ValidationError('Visit can not be missing')
    else:
        ir.update(infant_visit=infant_visit)
        del ir['infant_visit_id']
    del ir['export_change_type']
    del ir['is_receive_datetime']
    del ir['export_uuid']
    del ir['exported']
    del ir['exported_datetime']
    del ir['_infant_visit_cache']
    del ir['is_packed']
    del ir['is_receive']
    del ir['aliquot_type_id']
    del ir['is_labelled_datetime']
    del ir['study_site']
    del ir['panel_id']
    del ir['protocol']
    del ir['packing_list_id']
    del ir['is_lis']
    del ir['item_count_total']
    del ir['is_labelled']
    del ir['_panel_cache']
    del ir['specimen_identifier']
    if ir['panel_name'] == 'PBMC Plasma (STORE ONLY)':
        ir.update(panel_name='infant_pbmc_pl_store')
    elif ir['panel_name'] == 'DNA PCR':
        ir.update(panel_name='dna_pcr')
    elif ir['panel_name'] == 'Infant Insulin':
        ir.update(panel_name='infant_insulin')
    elif ir['panel_name'] == 'Infant PBMC PL':
        ir.update(panel_name='infant_pbmc_pl')
    elif ir['panel_name'] == 'DBS (Store Only)':
        ir.update(panel_name='dbs')
    elif ir['panel_name'] == 'Infant Glucose':
        ir.update(panel_name='infant_glucose')
    elif ir['panel_name'] == 'Infant Serum (Store Only)':
        ir.update(panel_name='serum_storage')
    elif ir['panel_name'] == 'ELISA':
        ir.update(panel_name='infant_elisa')
    try:
        panel = Panel.objects.get(name=ir['panel_name'])
    except Panel.DoesNotExist:
        print(ir['panel_name'])
        raise ValidationError('Panel can not be missing')
    else:
        ir.update(panel=panel)
        del ir['panel_name']
    try:
        InfantRequisition.objects.get(id=ir.get('id'))
    except InfantRequisition.DoesNotExist:
        infant_requisition = InfantRequisition(**ir)
        infant_requisition.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created infant requisition {count}')
print(f'Already created {already_created}')
print("Expected: ", len(ir_data))
print("Existing: ", InfantRequisition.objects.all().count())

Total created infant requisition 5632
Already created 0
Expected:  5632
Existing:  5632


In [22]:
from td_maternal.models import TdConsentVersion, SubjectScreening, SubjectConsent

count = 0
for si in SubjectScreening.objects.all():
    try:
        TdConsentVersion.objects.get(screening_identifier=si.screening_identifier)
    except TdConsentVersion.DoesNotExist:
        consent = SubjectConsent.objects.filter(
            subject_identifier=si.subject_identifier).order_by('created').last()
        if consent:
            td_consent_version = TdConsentVersion.objects.create(
                version=consent.version,
                screening_identifier=si.screening_identifier,
                report_datetime=si.created
            )
            count += 1
print('Total created is: ', count)

Total created is:  97


In [23]:
from td_maternal.models import SubjectConsent
from td_prn.models import MaternalOffStudy
from edc_action_item.models import ActionItem
from datetime import timedelta

fname = path + 'maternal_off_study.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]
offstudy_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    offstudy_data.append(data)

gaborone = Site.objects.get(id=40)
already_created = 0
count = 0

for off in offstudy_data:
    del off['maternal_visit_id']
    del off['_state']
    del off['_maternal_visit_cache']
    del off['consent_version']
    del off['export_change_type']
    del off['export_uuid']
    del off['exported']
    del off['exported_datetime']
    try:
        subject_consent = SubjectConsent.objects.filter(subject_identifier=off.get('subject_identifier')).order_by('-created').last()
    except SubjectConsent.DoesNotExist:
        raise ValidationError('Missing consent for ', off.get('subject-identifier'))
    else:
        off.update(site=subject_consent.site)
    if off['modified']:
        off.update(modified=make_aware(parser.parse(off.get('modified'))))
    else:
        off.update(modified=None)
    if off['offstudy_date']:
        off.update(offschedule_datetime=make_aware(parser.parse(off.get('created'))) + timedelta(hours=23))
    else:
        off.update(offschedule_datetime=None)
    if off['offstudy_date']:
        off.update(offstudy_date=make_aware(parser.parse(off.get('offstudy_date'))).date())
    else:
        off.update(offstudy_date=None)
    off.update(
        created=make_aware(parser.parse(off.get('created'))),
        id=uuid.UUID(off.get('id')),
    )
    try:
        of = MaternalOffStudy.objects.get(subject_identifier=off.get('subject_identifier'))
    except MaternalOffStudy.DoesNotExist:
        subject_offstudy = MaternalOffStudy(**off)
        subject_offstudy.save()
        count += 1
    else:
        already_created += 1
print(f'Total created Subject OffStudy: ', count)
print(f'Already created {already_created}')
print("Expected: ", len(offstudy_data))
print("Existing: ", MaternalOffStudy.objects.all().count())

Total created Subject OffStudy:  125
Already created 0
Expected:  125
Existing:  125


In [ ]:
from td_maternal.models import MaternalLabourDel
from edc_registration.models import RegisteredSubject
from django.apps import apps as django_apps
from datetime import datetime

mlds = MaternalLabourDel.objects.all()
count = 0
missing_rs = []
for mld in mlds:
    maternal_consent = SubjectConsent.objects.filter(
        subject_identifier=mld.subject_identifier
    ).order_by('version').last()
    try:
        registered_subject = RegisteredSubject.objects.get(
            relative_identifier=mld.subject_identifier)
    except RegisteredSubject.DoesNotExist:
        missing_rs.append(mld.subject_identifier)
    else:
        infant_consent_model_cls = django_apps.get_model('td_infant.infantdummysubjectconsent')
        try:
            infant_consent_model_cls.objects.get(
                subject_identifier=registered_subject.subject_identifier)
        except infant_consent_model_cls.DoesNotExist:
            old_consent_version_date = make_aware(datetime(2018, 2, 1, 0, 0, 0))
            if mld.created <= old_consent_version_date:
                version = '1'
            else:
                version = maternal_consent.version
            infant_consent_model_cls.objects.create(
                subject_identifier=registered_subject.subject_identifier,
                version=version,
                report_datetime=mld.created,
                consent_datetime=mld.created
            )
            count += 1
print('Total created: ', count)

In [ ]:
from td_maternal.models import MaternalLabourDel
count = 0
for rs in RegisteredSubject.objects.all():
    if rs.subject_identifier[-3:] == '-10':
        try:
            mld = MaternalLabourDel.objects.get(subject_identifier=rs.relative_identifier)
        except MaternalLabourDel.DoesNotExist:
            print('Missing', rs.relative_identifier)
        else:
            if not rs.consent_datetime:
                rs.consent_datetime = mld.created
                rs.save_base(raw=True)
                count += 1
print("Updated: ", count)

In [ ]:
from td_prn.models import InfantOffStudy
from edc_action_item.models import ActionItem
from datetime import timedelta

fname = path + 'infant_off_study.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
lines.pop(0)
del headers[0]
infant_offstudy_data = []
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(line)
        print('line: ', len(line))
        print('headers: ', len(headers))
        raise ValidationError('Line is not the right size')
    data = dict(zip(headers, line))
    infant_offstudy_data.append(data)

gaborone = Site.objects.get(id=40)
already_created = 0
count = 0

for ioff in infant_offstudy_data:
    del ioff['infant_visit_id']
    del ioff['_state']
    del ioff['_infant_visit_cache']
    del ioff['export_change_type']
    del ioff['export_uuid']
    del ioff['exported']
    del ioff['exported_datetime']
    ioff.update(site=gaborone)
    if ioff['modified']:
        ioff.update(modified=make_aware(parser.parse(ioff.get('modified'))))
    else:
        ioff.update(modified=None)
    if ioff['offstudy_date']:
        ioff.update(offschedule_datetime=make_aware(parser.parse(ioff.get('created'))) + timedelta(hours=23))
    else:
        ioff.update(offschedule_datetime=None)
    if ioff['offstudy_date']:
        ioff.update(offstudy_date=make_aware(parser.parse(ioff.get('offstudy_date'))).date())
    else:
        ioff.update(offstudy_date=None)
    ioff.update(
        created=make_aware(parser.parse(ioff.get('created'))),
        id=uuid.UUID(ioff.get('id')),
    )
    try:
        InfantOffStudy.objects.get(subject_identifier=ioff.get('subject_identifier'))
    except InfantOffStudy.DoesNotExist:
        infant_offstudy = InfantOffStudy(**ioff)
        infant_offstudy.save()
        count += 1
    else:
        already_created += 1
print(f'Total created Infanr OffStudy: ', count)
print(f'Already created {already_created}')
print("Expected: ", len(infant_offstudy_data))
print("Existing: ", InfantOffStudy.objects.all().count())

In [ ]:
# Import CRFs
import csv
from pprint import pprint
from decimal import Decimal
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
crf_list = ['maternalarvpostmed', 'maternalarv']
data_path = path + 'maternal_inlines/'

problem = []
for crf_name in crf_list:
    fname = data_path + crf_name + '.csv'
    with open(fname, encoding="ISO-8859-1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        header = next(csv_reader)
        del header[0]
        created = 0
        already_exists = 0
        count = 0
        for line in csv_reader:
            del line[0]
            if len(line) == len(header):
                data = dict(zip(header, line))
            else:
                print(lines[count - 1])
                print(line)
                print(lines[count + 1])
                raise ValidationError('Line is not the right size')
            data.update(
            created=make_aware(parser.parse(data.get('created'))),
            id=uuid.UUID(data.get('id')))
            if data['modified']:
                data.update(
                    modified=make_aware(parser.parse(data.get('modified'))))
            else:
                data.update(modified=None)
            if 'maternalarv' == crf_name:
                del data['export_change_type']
                del data['export_uuid']
                del data['exported']
                del data['exported_datetime']
                if data['start_date']:
                    data.update(
                        start_date=make_aware(parser.parse(data.get('start_date'))).date())
                else:
                    data.update(start_date=None)
                if data['stop_date']:
                    data.update(
                        stop_date=make_aware(parser.parse(data.get('stop_date'))).date())
                else:
                    data.update(stop_date=None)
            if 'maternalarvpostmed' == crf_name:
                if data['modification_date']:
                    data.update(
                        modification_date=make_aware(parser.parse(data.get('modification_date'))).date())
                else:
                    data.update(modification_date=None)
            del data['_state']
            crf_cls = django_apps.get_model('td_maternal', crf_name)
            try:
                crf_cls.objects.get(id=data.get('id'))
            except crf_cls.DoesNotExist:
                try:
                    obj = crf_cls(**data)
                    obj.save_base(raw=True)
                except IntegrityError as e:
                    print(e)
                    problem.append({crf_name: data})
                else:
                    created += 1
            else:
                already_exists += 1
        print('**************************************************')
        print(f'Created: {created} for {crf_name}')
        print(f'Already Exists: {already_exists} for {crf_name}')
        print('###################################################')
print('Done woth all CRFS')

In [ ]:
# Import CRFs
import csv
from pprint import pprint
from decimal import Decimal
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
crf_list = [
    'infantvaccines',  
    'infantcns',
    'infantfacialdefect',
    'infantcleftdisorder',
    'infantmouthupgi',
    'infantcardiodisorder',
    'infantrespiratorydefect',
    'infantlowergi',
    'infantfemalegenital',
    'infantmalegenital',
    'infantrenal',
    'infantmusculoskeletal',
    'infantskin',
    'infanttrisomies',
    'infantotherabnormalityitems',
    'infantfudxitems',
    'vaccinesreceived',  
    'vaccinesmissed', 
    'infantfunewmeditems',
    'infantarvprophmod'
]
data_path = path + 'infant_inlines/'

problem = []
for crf_name in crf_list:
    fname = data_path + crf_name + '.csv'
    with open(fname, encoding="ISO-8859-1") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        header = next(csv_reader)
        del header[0]
        created = 0
        already_exists = 0
        count = 0
        for line in csv_reader:
            del line[0]
            if len(line) == len(header):
                data = dict(zip(header, line))
            else:
                print(lines[count - 1])
                print(line)
                print(lines[count + 1])
                raise ValidationError('Line is not the right size')
            data.update(
            created=make_aware(parser.parse(data.get('created'))),
            id=uuid.UUID(data.get('id')))
            if data['modified']:
                data.update(
                    modified=make_aware(parser.parse(data.get('modified'))))
            else:
                data.update(modified=None)
            if 'infantarvprophmod' == crf_name:
                if data['modification_date']:
                    data.update(
                        modification_date=make_aware(parser.parse(data.get('modification_date'))).date())
                else:
                    data.update(modification_date=None)
            if 'infantvaccines' == crf_name:
                if data['vaccine_date']:
                    data.update(
                        vaccine_date=make_aware(parser.parse(data.get('vaccine_date'))).date())
                else:
                    data.update(vaccine_date=None)
                del data['export_change_type']
                del data['export_uuid']
                del data['exported']
                del data['exported_datetime']
            if 'vaccinesreceived' == crf_name:
                if data['date_given']:
                    data.update(
                        date_given=make_aware(parser.parse(data.get('date_given'))).date())
                else:
                    data.update(date_given=None)
            
            if 'infantfunewmeditems' == crf_name:
                if data['date_first_medication']:
                    data.update(
                        date_first_medication=make_aware(parser.parse(data.get('date_first_medication'))).date())
                else:
                    data.update(date_first_medication=None)
                if data['stop_date']:
                    data.update(
                        stop_date=make_aware(parser.parse(data.get('stop_date'))).date())
                else:
                    data.update(stop_date=None)
            if crf_name in ['infantmusculoskeletal', 'infantlowergi', 'infantrespiratorydefect', 'infantcns', 'infantskin', 'infantfudxitems', 'infantfunewmeditems', 'infantarvprophmod']:
                del data['export_change_type']
                del data['export_uuid']
                del data['exported']
                del data['exported_datetime']
            del data['_state']
            crf_cls = django_apps.get_model('td_infant', crf_name)
            try:
                crf_cls.objects.get(id=data.get('id'))
            except crf_cls.DoesNotExist:
                try:
                    obj = crf_cls(**data)
                    obj.save_base(raw=True)
                except IntegrityError as e:
                    print(e)
                    problem.append({crf_name: data})
                else:
                    created += 1
            else:
                already_exists += 1
        print('**************************************************')
        print(f'Created: {created} for {crf_name}')
        print(f'Already Exists: {already_exists} for {crf_name}')
        print('###################################################')
print('Done woth all CRFS')

In [ ]:
# RandomizationItem
import uuid
from td_maternal.models.list_models import RandomizationItem
from django.core.exceptions import ValidationError
from pprint import pprint

fname = path + 'randomization_item.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

rando_item_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    rando_item_data.append(data)
    count += 0

count = 0
already_created = 0
for ri in rando_item_data:
    del ri['_state']
    created = parser.parse(ri.get('created'))
    created = make_aware(created)
    ri.update(
        created=created,
        id=uuid.UUID(ri.get('id')),
    )
    if ri['modified']:
        modified = parser.parse(ri.get('modified'))
        modified = make_aware(modified)
        ri.update(modified=modified)
    else:
        ri.update(modified=None)
    try:
        RandomizationItem.objects.get(id=ri.get('id'))
    except RandomizationItem.DoesNotExist:
        rando_item = RandomizationItem(**ri)
        rando_item.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created RandomizationItem {count}')
print(f'Already created RandomizationItem {already_created}')
print("Expected: ", len(rando_item_data))
print("Existing: ", RandomizationItem.objects.all().count())

In [ ]:
# Maternal Contact
import uuid
from td_maternal.models import MaternalContact
from django.core.exceptions import ValidationError
from pprint import pprint

fname = path + 'maternal_contact.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

maternal_contact_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    maternal_contact_data.append(data)
    count += 0

count = 0
already_created = 0
for mt in maternal_contact_data:
    del mt['_state']
    del mt['registered_subject_id']
    del mt['_registered_subject_cache']
    created = parser.parse(mt.get('created'))
    created = make_aware(created)
    mt.update(
        created=created,
        id=uuid.UUID(mt.get('id')),
    )
    if mt['modified']:
        modified = parser.parse(mt.get('modified'))
        modified = make_aware(modified)
        mt.update(modified=modified)
    else:
        mt.update(modified=None)
    if mt['contact_datetime']:
        contact_datetime = parser.parse(mt.get('contact_datetime'))
        contact_datetime = make_aware(contact_datetime)
        mt.update(contact_datetime=contact_datetime)
    else:
        mt.update(contact_datetime=mt['created'])
    
    if mt['report_datetime']:
        report_datetime = parser.parse(mt.get('report_datetime'))
        report_datetime = make_aware(report_datetime)
        mt.update(report_datetime=contact_datetime)
    else:
        mt.update(report_datetime=None)
    try:
        MaternalContact.objects.get(id=mt.get('id'))
    except MaternalContact.DoesNotExist:
        maternal_contact = MaternalContact(**mt)
        maternal_contact.save_base(raw=True)
        count += 1
    else:
        already_created += 1
print(f'Total created MaternalContact {count}')
print(f'Already created MaternalContact {already_created}')
print("Expected: ", len(maternal_contact_data))
print("Existing: ", MaternalContact.objects.all().count())

In [ ]:
#load crf many to many data
import csv
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
import uuid
fname = path + 'df_maternal_crf_list.csv'
problematic = []
something_done = False
with open(fname, encoding="ISO-8859-1") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for line in csv_reader:
        del line[0]
        crf, obj_id, field_name, m_name_cls, m_data = line
        crf_cls = django_apps.get_model('td_maternal', crf)
        m_data = m_data.split('|')
        del m_data[-1]
        try:
            crf_obj = crf_cls.objects.get(id=uuid.UUID(obj_id))
        except crf_cls.DoesNotExist:
            pass
        else:
            m_cls = django_apps.get_model('td_maternal', m_name_cls)
            for mm in m_data:
                if mm == 'Pre-eclampsia/eclampsia-Pre-eclampsia/eclampsia':
                    short_name = mm[:23]
                    name = mm[-23:]
                else:
                    mm = mm.split('-')
                    short_name, name = mm
                try:
                    mobj = m_cls.objects.get(name=name)
                except m_cls.DoesNotExist:
                    problematic.append([crf_obj.id, crf, m_name_cls, field_name, short_name, name])
                except exception as e:
                    print(e)
                else:
                    m_obj_d = getattr(crf_obj, field_name)
                    something_done = False
                    if field_name == 'prior_arv':
                        if not crf_obj.prior_arv.filter(name=name):
                            crf_obj.prior_arv.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'contraceptive_relative':
                        if not crf_obj.contraceptive_relative.filter(name=name):
                            crf_obj.contraceptive_relative.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'contr':
                        if not crf_obj.contr.filter(name=name):
                            crf_obj.contr.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'delivery_complications':                         
                        if not crf_obj.delivery_complications.filter(name=name):
                            crf_obj.delivery_complications.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'who':
                        if not crf_obj.who.filter(name=name):
                            crf_obj.who.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'mother_chronic':
                        if not crf_obj.mother_chronic.filter(name=name):
                            crf_obj.mother_chronic.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'father_chronic':
                        if not crf_obj.father_chronic.filter(name=name):
                            crf_obj.father_chronic.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'mother_medications':
                        if not crf_obj.mother_medications.filter(name=name):
                            crf_obj.mother_medications.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'hospitalization_reason':
                        if not crf_obj.hospitalization_reason.filter(name=name):
                            crf_obj.hospitalization_reason.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
            if something_done:
                crf_obj.save_base(raw=True)
                print('************************', crf, 'Done', '**************************')
print("Done")

In [ ]:
print(len(problematic))
problematic

In [ ]:
#load crf many to many data
import csv
from django.apps import apps as django_apps
from django.core.handlers import exception
from django.db.utils import DataError
from django.db.utils import IntegrityError
from builtins import ValueError
import uuid
fname = path + 'df_infant_crf_list.csv'
problematic = []
something_done = False
with open(fname, encoding="ISO-8859-1") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for line in csv_reader:
        del line[0]
        crf, obj_id, field_name, m_name_cls, m_data = line
        crf_cls = django_apps.get_model('td_infant', crf)
        m_data = m_data.split('|')
        del m_data[-1]
        try:
            crf_obj = crf_cls.objects.get(id=uuid.UUID(obj_id))
        except crf_cls.DoesNotExist:
            pass
        else:
            m_cls = django_apps.get_model('td_infant', m_name_cls)
            for mm in m_data:
                mm = mm.split('-')
                short_name, name = mm
                try:
                    mobj = m_cls.objects.get(name=name)
                except m_cls.DoesNotExist:
                    if m_name_cls == 'Foods':
                        from td_infant.models.list_models import Foods
                        m_cls = Foods
                        mobj = m_cls.objects.get(short_name=short_name)
                        m_obj_d = getattr(crf_obj, field_name)
                        something_done = False
                        if field_name == 'solid_foods':
                            if not crf_obj.solid_foods.filter(name=name):
                                crf_obj.solid_foods.add(mobj)
                                something_done = True
                                print('CRF ', crf, 'and field: ', field_name, 'Updated')
                        elif field_name == 'rations_receviced':
                            if not crf_obj.rations_receviced.filter(name=name):
                                crf_obj.rations_receviced.add(mobj)
                                something_done = True
                                print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    else:
                        problematic.append([crf_obj.id, crf, m_name_cls, field_name, short_name, name])
                except exception as e:
                    print(e)
                else:
                    m_obj_d = getattr(crf_obj, field_name)
                    something_done = False
                    if field_name == 'solid_foods':
                        if not crf_obj.solid_foods.filter(name=name):
                            crf_obj.solid_foods.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
                    elif field_name == 'rations_receviced':
                        if not crf_obj.rations_receviced.filter(name=name):
                            crf_obj.rations_receviced.add(mobj)
                            something_done = True
                            print('CRF ', crf, 'and field: ', field_name, 'Updated')
            if something_done:
                crf_obj.save_base(raw=True)
                print('************************', crf, 'Done', '**************************')
print("Done")

In [ ]:
problematic

In [ ]:
import csv
import uuid
fname = path + 'df_ap_status.csv'
missing = []
count = 0
with open(fname, encoding="ISO-8859-1") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for line in csv_reader:
        _, idd, appt_status, subject_identifier = line
        idd = uuid.UUID(idd)
        if subject_identifier[-3:] == '-10':
            from td_infant.models import Appointment
            try:
                ap = Appointment.objects.get(id=idd)
            except Appointment.DoesNotExist:
                missing.append([idd, appt_status, subject_identifier])
            else:
                ap.appt_status = appt_status
                ap.save_base(raw=True)
                count += 1
        else:
            from edc_appointment.models import Appointment
            try:
                ap = Appointment.objects.get(id=idd)
            except Appointment.DoesNotExist:
                missing.append([idd, appt_status, subject_identifier])
            else:
                ap.appt_status = appt_status
                ap.save_base(raw=True)
                count += 1
print(count, "Appointments updated.")
        

In [ ]:
print('Missing', len(missing))

In [ ]:
missing